In [1]:
import os
from os import listdir
import cv2
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa
import json
import pandas as pd
from pandas import json_normalize
from numpyencoder import NumpyEncoder
from tqdm import tqdm
from datetime import datetime
import warnings
import glob

## Loading Annotation file & Preprocsssing

In [4]:
ANNOTATION_FOLDER_PATH = '/opt/ml/detection/dataset/train.json'    # 기존 ann 파일 경로

with open(ANNOTATION_FOLDER_PATH, 'r') as f:
    train_json = json.load(f)
train_json = json_normalize(train_json)

train_ann = pd.DataFrame(columns=['image_id', 'category_id', 'area', 'bbox', 'iscrowd', 'id'])
train_img = pd.DataFrame(columns=['width', 'height', 'file_name', 'license', 'flickr_url', 'coco_url', 'date_captured', 'id'])

train_ann_raw = train_json.loc[0]['annotations']
train_img_raw = train_json.loc[0]['images']

for raw in tqdm(train_ann_raw, desc='extracting annotations...'):
    train_ann = train_ann.append(raw, ignore_index=True)
    
for raw in tqdm(train_img_raw, desc='extracting images info...'):
    train_img = train_img.append(raw, ignore_index=True)

extracting images info...: 100%|██████████| 4883/4883 [00:16<00:00, 298.70it/s]


In [5]:
# object별 annotation 정보 확인
train_ann

,image_id,category_id,area,bbox,iscrowd,id
0,0,0,257301.66,"[197.6, 193.7, 547.8, 469.7]",0,0
1,1,3,10402.56,"[0.0, 407.4, 57.6, 180.6]",0,1
2,1,7,26259.36,"[0.0, 455.6, 144.6, 181.6]",0,2
3,1,4,69096.17,"[722.3, 313.4, 274.3, 251.9]",0,3
4,1,5,24164.58,"[353.2, 671.0, 233.7, 103.4]",0,4
...,...,...,...,...,...,...
23139,4882,5,768591.81,"[0.0, 116.2, 944.1, 814.1]",0,23139
23140,4882,7,57309.72,"[302.1, 439.3, 265.2, 216.1]",0,23140
23141,4882,0,1772.74,"[511.3, 451.1, 58.7, 30.2]",0,23141
23142,4882,1,53008.67,"[255.0, 421.4, 271.7, 195.1]",0,23142


In [6]:
# image별 정보 확인
train_img

,width,height,file_name,license,flickr_url,coco_url,date_captured,id
0,1024,1024,train/0000.jpg,0,None,None,2020-12-26 14:44:23,0
1,1024,1024,train/0001.jpg,0,None,None,2021-01-10 16:30:39,1
2,1024,1024,train/0002.jpg,0,None,None,2020-12-27 17:55:52,2
3,1024,1024,train/0003.jpg,0,None,None,2021-03-31 12:10:36,3
4,1024,1024,train/0004.jpg,0,None,None,2021-01-01 18:52:38,4
...,...,...,...,...,...,...,...,...
4878,1024,1024,train/4878.jpg,0,None,None,2021-01-04 14:36:54,4878
4879,1024,1024,train/4879.jpg,0,None,None,2021-04-27 13:46:18,4879
4880,1024,1024,train/4880.jpg,0,None,None,2020-12-24 11:25:46,4880
4881,1024,1024,train/4881.jpg,0,None,None,2021-01-05 11:01:18,4881


## Loading existing images

In [3]:
IMAGE_FOLDER_PATH = '/opt/ml/detection/dataset/train'    # 기존 train 폴더 경로
images = []

for file in tqdm(sorted(listdir(IMAGE_FOLDER_PATH)), desc='getting images...'):
    images.append(cv2.imread(os.path.join(IMAGE_FOLDER_PATH, file), 1))
images = np.array(images)

getting images...: 100%|██████████| 4883/4883 [01:16<00:00, 63.87it/s]


## Augmentation

In [7]:
# SAVE_DIR_IMAGES = '/opt/ml/detection/dataset/new_train/'    # 테스트 폴더
SAVE_DIR_IMAGES = '/opt/ml/detection/dataset/train/'    # 실제 폴더
SAVE_DIR_ANNOTATIONS = '/opt/ml/detection/dataset/new_train.json'    # 저장할 json 파일 경로
SAVE_DATE = datetime.today().strftime('%Y-%m-%d %H:%M:%S')

# 기존 이미지 4882 뒤에, 기존 bbox 23143 뒤에 새로운 정보들 추가
new_img_idx = 4882
new_object_idx = 23143

In [8]:
# imgaug 설명 및 예시: https://uos-deep-learning.tistory.com/17

seq = iaa.Sequential([
        iaa.Fliplr(0.8),    # horizontally flip 80% of all images
        iaa.Flipud(0.8),    # vertically flip 80% of all images
        ])
seq_det = seq.to_deterministic()

In [9]:
ia.seed(1)

for idx in tqdm(range(len(images)), desc='Augmentation...'):
    image = images[idx]
    boxes = train_ann[train_ann['image_id']==idx]
    category = list(boxes['category_id'])
    
    ia_bounding_boxes = []
    for df_idx, df_rows in boxes.iterrows():
        ia_bounding_boxes.append(ia.BoundingBox(x1=df_rows.bbox[0], 
                                                y1=df_rows.bbox[1], 
                                                x2=df_rows.bbox[0]+df_rows.bbox[2], 
                                                y2=df_rows.bbox[1]+df_rows.bbox[3]))
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)
    
    image_aug = seq_det.augment_images([image])[0]    # 이미지 aug 적용
    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]    # bbox aug 적용
    
    new_img_idx += 1
    new_image_file = SAVE_DIR_IMAGES + str(new_img_idx) + '.jpg'
    
    # 이미지, bbox, image 정보 수정 및 저장
    cv2.imwrite(new_image_file, image_aug)
    
    new_train_img = {'width':1024, 
                     'height':1024, 
                     'file_name':f'train/{new_img_idx}.jpg', 
                     'license':0, 
                     'flickr_url':None, 
                     'coco_url':None, 
                     'date_captured':SAVE_DATE,
                     'id':new_img_idx
                    }
    train_img = train_img.append(new_train_img, ignore_index=True)
    
    for bbs_idx in range(0, len(bbs_aug)):
        new_object_idx += 1
        
        x_min = round(bbs_aug[bbs_idx].x1, 1)
        y_min = round(bbs_aug[bbs_idx].y1, 1)
        width = round(bbs_aug[bbs_idx].x2-bbs_aug[bbs_idx].x1, 1)
        height = round(bbs_aug[bbs_idx].y2-bbs_aug[bbs_idx].y1, 1)
        new_train_ann = {'image_id':new_img_idx,
                         'category_id':category[bbs_idx],
                         'area':width*height,
                         'bbox':[x_min, y_min, width, height],
                         'iscrowd':0,
                         'id':new_object_idx}
        train_ann = train_ann.append(new_train_ann, ignore_index=True)

Augmentation...: 100%|██████████| 4883/4883 [04:33<00:00, 17.84it/s]


## Save Modified Annotations

In [10]:
# 수정된 object별 annotation 정보 확인 (23144*2=46288)
train_ann

,image_id,category_id,area,bbox,iscrowd,id
0,0,0,257301.66,"[197.6, 193.7, 547.8, 469.7]",0,0
1,1,3,10402.56,"[0.0, 407.4, 57.6, 180.6]",0,1
2,1,7,26259.36,"[0.0, 455.6, 144.6, 181.6]",0,2
3,1,4,69096.17,"[722.3, 313.4, 274.3, 251.9]",0,3
4,1,5,24164.58,"[353.2, 671.0, 233.7, 103.4]",0,4
...,...,...,...,...,...,...
46283,9765,5,768591.81,"[79.9, 116.2, 944.1, 814.1]",0,46283
46284,9765,7,57309.72,"[456.7, 439.3, 265.2, 216.1]",0,46284
46285,9765,0,1772.74,"[454.0, 451.1, 58.7, 30.2]",0,46285
46286,9765,1,53008.67,"[497.3, 421.4, 271.7, 195.1]",0,46286


In [11]:
# 수정된 image별 정보 확인 (4883*2=9766)
train_img

,width,height,file_name,license,flickr_url,coco_url,date_captured,id
0,1024,1024,train/0000.jpg,0,None,None,2020-12-26 14:44:23,0
1,1024,1024,train/0001.jpg,0,None,None,2021-01-10 16:30:39,1
2,1024,1024,train/0002.jpg,0,None,None,2020-12-27 17:55:52,2
3,1024,1024,train/0003.jpg,0,None,None,2021-03-31 12:10:36,3
4,1024,1024,train/0004.jpg,0,None,None,2021-01-01 18:52:38,4
...,...,...,...,...,...,...,...,...
9761,1024,1024,train/9761.jpg,0,None,None,2022-04-02 13:39:39,9761
9762,1024,1024,train/9762.jpg,0,None,None,2022-04-02 13:39:39,9762
9763,1024,1024,train/9763.jpg,0,None,None,2022-04-02 13:39:39,9763
9764,1024,1024,train/9764.jpg,0,None,None,2022-04-02 13:39:39,9764


In [12]:
# 기존 json 파일 형태 확인
train_json

,licenses,images,categories,annotations,info.year,info.version,info.description,info.contributor,info.url,info.date_created
0,"[{'id': 0, 'name': 'CC BY 4.0', 'url': 'https:...","[{'width': 1024, 'height': 1024, 'file_name': ...","[{'id': 0, 'name': 'General trash', 'supercate...","[{'image_id': 0, 'category_id': 0, 'area': 257...",2021,1.0,Recycle Trash,Upstage,None,2021-02-02 01:10:00


In [13]:
# 기존 json 형태대로 수정된 ann 저장
new_train_json = train_json.copy()

new_train_json.at[0, 'images'] = train_img.to_dict('records')
new_train_json.at[0, 'annotations'] = train_ann.to_dict('records')

with open(SAVE_DIR_ANNOTATIONS, 'w') as outfile:
    json.dump(new_train_json.to_dict('records')[0], outfile, cls=NumpyEncoder, indent=4)